# Doğuş Datathon - Zingat / Coderspace

In [ ]:
import pandas as pd
import numpy as np
from pandas import Timestamp
from ggplot import *
import pandas
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from matplotlib import pyplot
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
from scipy import stats
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})

train = pd.read_csv("Adsızz.csv")
real_test = pd.read_csv("real_test2.csv")

train.iloc[:,[0,1]].head()
for col in train.columns:
    print(col)


In [ ]:
train.describe(include='all')
real_test.describe(include='all')

# Data Preprocessing
Data Cleaning

In [ ]:
#Binning Categorical Features 

train.path = train.path.apply(lambda x: 'Istanbul' if 'Istanbul' in x else x)    
train.path = train.path.apply(lambda x: 'Izmir' if 'Izmir' in x else x) 

counts = train["path"].value_counts()
percent = train["path"].value_counts(normalize=True)
percent100 = train["path"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.DataFrame({'counts': counts, 'per': percent, 'per100': percent100})

#Binning Categorical Features

real_test.path = real_test.path.apply(lambda x: 'Istanbul' if 'Istanbul' in x else x)    
real_test.path = real_test.path.apply(lambda x: 'Izmir' if 'Izmir' in x else x) 

counts = real_test["path"].value_counts()
percent = real_test["path"].value_counts(normalize=True)
percent100 = real_test["path"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.DataFrame({'counts': counts, 'per': percent, 'per100': percent100})

In [ ]:
tarih_epoch = pd.to_datetime(train['tarih']) - pd.datetime(2018, 1, 1)
train['tarih'] = tarih_epoch.dt.days

tarih_epoch = pd.to_datetime(real_test['tarih']) - pd.datetime(2018, 1, 1)
real_test['tarih'] = tarih_epoch.dt.days

Missing Value Imputation

In [ ]:
nulls = pd.DataFrame(train.isnull().sum().sort_values(ascending=False)[:25])
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
nulls

nulls2 = pd.DataFrame(real_test.isnull().sum().sort_values(ascending=False)[:25])
nulls2.columns = ['Null Count']
nulls2.index.name = 'Feature'
nulls2

In [ ]:
del train["manzara"]
del train["otopark"]
train = train.fillna(train.mode().iloc[0])                 #Replace missing values with the most frequent value
nulls = pd.DataFrame(train.isnull().sum().sort_values(ascending=False)[:25])
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
nulls


del real_test["manzara"]
del real_test["otopark"]
real_test = real_test.fillna(real_test.mode().iloc[0])      #Replace missing values with the most frequent value
nulls2 = pd.DataFrame(real_test.isnull().sum().sort_values(ascending=False)[:25])
nulls2.columns = ['Null Count']
nulls2.index.name = 'Feature'
#nulls2

Removing and Handling Outliers

In [ ]:
num_train = train._get_numeric_data()
num_train.columns
def var_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  x.std(), x.var(), x.min(), x.quantile(0.01), x.quantile(0.05),x.quantile(0.10),x.quantile(0.25),x.quantile(0.50),x.quantile(0.75), x.quantile(0.90),x.quantile(0.95), x.quantile(0.99),x.max()], 
                  index=['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1' , 'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])

num_train.apply(lambda x: var_summary(x)).T

sns.boxplot(train['guncel_fiyat']) 
sns.boxplot(train['brutmkare']) 
sns.boxplot(train['netmkare']) 
train['guncel_fiyat']= train['guncel_fiyat'].clip(upper=train['guncel_fiyat'].quantile(0.99)) 
train['brutmkare']= train['brutmkare'].clip(upper=train['brutmkare'].quantile(0.99)) 
train['netmkare']= train['netmkare'].clip(upper=train['netmkare'].quantile(0.99)) 

Categorical Encoding

In [ ]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
train['path'] = labelencoder.fit_transform(train['path'])
train['emlaktipi'] = labelencoder.fit_transform(train['emlaktipi'])
train['isitmatipi'] = labelencoder.fit_transform(train['isitmatipi'])
real_test['path'] = labelencoder.fit_transform(real_test['path'])
real_test['emlaktipi'] = labelencoder.fit_transform(real_test['emlaktipi'])
real_test['isitmatipi'] = labelencoder.fit_transform(real_test['isitmatipi'])

le_enc = preprocessing.LabelEncoder()
le_enc = le_enc.fit(train["odasayisi"])
le_enc.classes_ = np.array(["1+0","1+1","2+0","2+1","2+2","3+1","3+2","4+1","4+2","4+3","4+4","5+1","5+2","5+3","6+1","6+2",
                            "6+3","7+1","7+2","7+3","8+1","8+2","8+3","8+4","9+1","9+2","9+3","9+4","10+0","10+1","10+2",
                            "10+3","10+4","1149+0","11+3","12+2","19+4","19+6"])
train["odasayisi"] = le_enc.transform(train["odasayisi"])
le_enc = preprocessing.LabelEncoder()
le_enc.classes_ = np.array(["1+0","1+1","2+0","2+1","2+2","3+1","3+2","4+1","4+2","4+3","4+4","5+1","5+2","5+3","6+1","6+2",
                            "6+3","7+1","7+2","7+3","8+1","8+2","8+3","8+4","9+1","9+2","9+3","9+4","10+0","10+1","10+2",
                            "10+3","10+4","1149+0","11+3","12+2","19+4","19+6"])
le_enc = le_enc.fit(real_test["odasayisi"])
real_test["odasayisi"] = le_enc.transform(real_test["odasayisi"])

#Find and Replace
preserved_mapper = {"bulundugukat": {'Kot 4':1 ,'Kot 3':1,'Kot 2':1,'Kot 1':1,'Bodrum Kat':1,'Zemin Kat':2,'Giris Kati':2,
                    'Bahce kati':2,'Yuksek Giris': 2,'Mustakil':2 ,'Komple':2, '1':3,'2':4,'3':5,'4':6,'5': 7, '6': 8,
                    '7': 9,'8': 10,'9': 11,'10': 12,'11': 13,'12': 14,'13': 15,'14': 16,'15': 17,'16': 18,'17': 19,'18': 20,
                    '19': 21,'20': 22,'20 ve uzeri': 23,'cati Kati': 24,'En ust Kat': 24,'Teras Kat': 24}}     
train = train.replace(preserved_mapper)  
real_test = real_test.replace(preserved_mapper)

preserved_mapper = {"binadakikatsayisi": {'1':1,'2':2,'3':3,'4':4,'5': 5, '6': 6, '7': 7,'8': 8,'9': 9,'10': 10,
                                          '10-20 arasi': 15,'20 ve uzeri': 25}}     
train = train.replace(preserved_mapper) 
real_test = real_test.replace(preserved_mapper)

preserved_mapper = {"binayasi": {'0':0,'1':1,'2':2,'3':3,'4':4,'5': 5,'6-10 arasi':8,'11-15 arasi':13,'16-20 arasi':18,
                                 '21-25 arasi':23,'26-30 arasi':28,'31-35 arasi':33,'36-40 arasi':38,'40 ve uzeri':45}}     
train = train.replace(preserved_mapper)
train.head() 
real_test = real_test.replace(preserved_mapper)
real_test.head()


In [ ]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

train = create_dummies(train,"emlaktipi")
real_test= create_dummies(train,"emlaktipi")
train = create_dummies(train,"path")
real_test= create_dummies(train,"path")

del train["emlaktipi"]
del train["path"]
del real_test["emlaktipi"]
del real_test["path"]

train.head()

In [ ]:
def encode(x):
 return 0 if x == 'YOK' else 1
train['Interkom'] = train["Interkom"].apply(encode)
real_test['Interkom'] = real_test["Interkom"].apply(encode)
def encode(x):
 return 0 if x == 'YOK' else 1
train['depremyonetmeligineuygun'] = train["depremyonetmeligineuygun"].apply(encode)
real_test['depremyonetmeligineuygun'] = real_test["depremyonetmeligineuygun"].apply(encode)
def encode(x):
 return 0 if x == 'YOK' else 1
train['asansor'] = train["asansor"].apply(encode)
real_test['asansor'] = real_test["asansor"].apply(encode)
def encode(x):
 return 0 if x == 'YOK' else 1
train['cocukoyunalani'] = train["cocukoyunalani"].apply(encode)
real_test['cocukoyunalani'] = real_test["cocukoyunalani"].apply(encode)
def encode(x):
 return 0 if x == 'YOK' else 1
train['giyinmeodasi'] = train["giyinmeodasi"].apply(encode)
real_test['giyinmeodasi'] = real_test["giyinmeodasi"].apply(encode)
def encode(x):
 return 0 if x == 'YOK' else 1
train['ebeveynbanyosu'] = train.ebeveynbanyosu.apply(encode)
real_test['ebeveynbanyosu'] = real_test.ebeveynbanyosu.apply(encode)

# Data Visualization

In [ ]:
pd.crosstab(index=train["brutmkare"],columns="count")

In [ ]:
categoricals = train.select_dtypes(exclude=[np.number])
categoricals.describe()
numeric_features = train.select_dtypes(include=[np.number])
numeric_features.describe()


In [ ]:
quality_pivot = train.pivot_table(index='banyosayisi',
                  values='guncel_fiyat', aggfunc=np.median)
quality_pivot.plot(kind='bar', color='blue')
plt.xlabel('banyosayisi')
plt.ylabel('guncel_fiyat')
plt.xticks(rotation=0)
plt.show()
plt.scatter(x=train['netmkare'], y=train['guncel_fiyat'])
plt.ylabel('guncel_fiyat')
plt.xlabel('netmkare')
plt.show()

train = train[train['netmkare'] < 120000] #We will create a new dataframe with some outliers removed.
plt.scatter(x=train['netmkare'], y=np.log(train["guncel_fiyat"]))
plt.xlim(-100,8000) # This forces the same scale as before
plt.ylabel("guncel_fiyat")
plt.xlabel('netmkare')
plt.show()

In [ ]:
condition_pivot = train.pivot_table(index='bulundugukat', values='guncel_fiyat', aggfunc=np.median)
condition_pivot.plot(kind='bar', color='blue')
plt.xlabel('bulundugukat',fontsize=3)
plt.ylabel('Median guncel_fiyat')
plt.xticks(rotation=0)
plt.show()

Correlation Plot

In [ ]:
k = 14
cols = num_corr.nlargest(k, 'guncel_fiyat')['guncel_fiyat'].index
cm = np.corrcoef(num_train[cols].values.T)
sns.set(font_scale=1.35)
f, ax = plt.subplots(figsize=(10,10))
hm=sns.heatmap(cm, annot = True,vmax =.8, yticklabels=cols.values, xticklabels = cols.values)

# Checking Normality

In [ ]:
plt.style.use(style='ggplot')
plt.rcParams['figure.figsize'] = (10, 6)
print ("Skew is:", train["guncel_fiyat"].skew())
plt.hist(train["guncel_fiyat"], color='blue')
plt.show()
fig = plt.figure()
ax1 = fig.add_subplot(211)
x = (train["guncel_fiyat"])
prob = stats.probplot(x, dist=stats.norm, plot=ax1)
ax1.set_xlabel('')
ax1.set_title('Probplot against normal distribution')


In [ ]:
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error



import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

y =np.log(train['guncel_fiyat'])
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=stats.johnsonsu)
plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=stats.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=stats.lognorm)

# Handling Multi-Collinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=10.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]
calculate_vif_(train[features])


In [ ]:
# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)
vif_X = train[features]
calc_vif(vif_X)
vif_X = train.drop(['netmkare','isitmatipi','guncel_fiyat','banyosayisi','emlak','tarih'], axis=1)
calc_vif(vif_X)



In [ ]:
train = train.drop(['netmkare','isitmatipi','guncel_fiyat','banyosayisi','emlak','tarih'], axis=1)
real_test = real_test.drop(['netmkare','isitmatipi','guncel_fiyat','banyosayisi','emlak','tarih'], axis=1)
train.reset_index(drop=True, inplace=True)
real_test.reset_index(drop=True, inplace=True)

# Forecasting

In [ ]:
train["guncel_fiyat"] = np.log1p(train["guncel_fiyat"])               #normalization
y = train['guncel_fiyat'].reset_index(drop=True)
X_train = train.drop(['guncel_fiyat'], axis=1)
X_test = real_test.drop(['guncel_fiyat'], axis=1)

In [ ]:
overfit = []                                                          #overfitting
for i in X.columns:
    counts = X[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(X) * 100 > 99.94:
        overfit.append(i)

overfit = list(overfit)
X = X_train.drop(overfit, axis=1)
X_test = X_test.drop(overfit, axis=1)

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, alphas=alphas2, random_state=42, cv=kfolds))
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))                                
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003,))
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42) 

xgboost = XGBRegressor(learning_rate=0.1,n_estimators=1000,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

In [ ]:
score = cv_rmse(ridge)
print("ridge: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lasso)
print("lasso: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(elasticnet)
print("elastic net: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(gbr)
print("gbr: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(xgboost)
print("xgboost: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [ ]:
print('rf')
rf_reg = RandomForestRegressor(n_estimators=1000)
rf_model_full_data = rf_reg.fit(X, y)

print('ridge')
ridge_model_full_data = ridge.fit(X, y)

print('elasticnet')
elastic_model_full_data = elasticnet.fit(X, y)

print('Lasso')
lasso_model_full_data = lasso.fit(X, y)

print('GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)

print('xgboost')
xgb_model_full_data = xgboost.fit(X, y)

In [ ]:
print('RMSLE score on train data:')
print(rmsle(y, rf_model_full_data.predict(X)))                 # best prediction
print('RMSLE score on train data:')
print(rmsle(y, ridge_model_full_data.predict(X)))
print('RMSLE score on train data:')
print(rmsle(y, elastic_model_full_data.predict(X)))
print('RMSLE score on train data:')
print(rmsle(y, gbr_model_full_data.predict(X)))
print('RMSLE score on train data:')
print(rmsle(y, xgb_model_full_data.predict(X)))

In [ ]:
np.floor(np.expm1(rf_model_full_data.predict(X_test))).head()